# 03_ 주가 관련 보조지표를 활용하여 등락 예측_카카오

In [1]:
# 기본적으로 필요한 라이브러리
import numpy as np
import pandas as pd


#시각화 라이브러리
import matplotlib.pyplot as plt 
import seaborn as sns

#datetime 형식으로 바꿔주는 라이브러리
from datetime import datetime

In [2]:
stock = pd.read_csv('kakao.csv')
stock.head()

,Date,Open,High,Low,Close,Volume
0,2020-09-01,82700,83200,80000,80300,5503680
1,2020-09-02,81100,82500,80300,82400,4608275
2,2020-09-03,83100,83200,80800,82000,3696075
3,2020-09-04,78000,80900,77900,80400,5842645
4,2020-09-07,80300,80300,78200,78400,4644695


In [3]:
stock = stock.set_index('Date') # date를 index로 설정
stock = stock.loc['2020-09-01' : '2021-06-30']

## 1.1 머신러닝 준비

In [4]:
# 종가 기준 fluctuation 추가
def up_down(x):
    if x >= 0:
        return 1
    else :
        return 0

stock['fluctuation'] = (stock['Close'].shift(-1)-stock['Close']).apply(up_down)
stock.head()

,Open,High,Low,Close,Volume,fluctuation
Date,,,,,,
2020-09-01,82700,83200,80000,80300,5503680,1
2020-09-02,81100,82500,80300,82400,4608275,0
2020-09-03,83100,83200,80800,82000,3696075,0
2020-09-04,78000,80900,77900,80400,5842645,0
2020-09-07,80300,80300,78200,78400,4644695,0


In [5]:
stock_train = stock.iloc[:-1]
target = stock_train['fluctuation']
stock_train = stock_train.drop('fluctuation', axis = 1)

from sklearn.model_selection import train_test_split
train, test, train_target, test_target = train_test_split(
    stock_train, target, test_size=0.3, shuffle=False)

## 1.2 모델링

In [6]:
# Importing Classifier Modules
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.linear_model import LogisticRegression   # logistic
from sklearn.neighbors import KNeighborsClassifier   # knn
from sklearn.naive_bayes import GaussianNB   # naive
from lightgbm import LGBMClassifier   # lightGBM
from sklearn import datasets
from sklearn import metrics

In [7]:
# 모형 생성
import lightgbm as lgb
logistic = LogisticRegression()
knn = KNeighborsClassifier()
naive = GaussianNB()
lgb = lgb.LGBMClassifier()

models = [{'name' : 'Logistic', 'model' : logistic}, {'name' : 'KNN', 'model' : knn},
          {'name' : 'lgb.LGBM', 'model' : lgb}, {'name' : 'NaiveBayes', 'model' : naive}]

for m in models : 
    model = m['model']
    model.fit(train, train_target)
    
    predicted = model.predict(test)

#Accuracy : 전체 샘플 중 맞게 예측한 샘플수의 비율
#Precision(정밀도) : postive라고 예측한 것 중에서 실제 postive인 것
#Recall(재현율) : 실제 postive중에 예측한 postive 비율
    print ('model name : {}'.format(m['name']))
    print (metrics.classification_report(test_target, predicted))

#confusion_matrix에서
#행은 실제값, 열은 예측한 값으로 0 1 순서대로 임
    print('Confusion Matrix') 
    print (metrics.confusion_matrix (test_target, predicted))

    print ('Accuracy Score : {:.4f}\n'.format(metrics.accuracy_score(test_target, predicted)))

model name : Logistic
              precision    recall  f1-score   support

           0       0.19      0.30      0.24        20
           1       0.55      0.40      0.47        42

    accuracy                           0.37        62
   macro avg       0.37      0.35      0.35        62
weighted avg       0.43      0.37      0.39        62

Confusion Matrix
[[ 6 14]
 [25 17]]
Accuracy Score : 0.3710

model name : KNN
              precision    recall  f1-score   support

           0       0.16      0.20      0.18        20
           1       0.57      0.50      0.53        42

    accuracy                           0.40        62
   macro avg       0.36      0.35      0.35        62
weighted avg       0.44      0.40      0.42        62

Confusion Matrix
[[ 4 16]
 [21 21]]
Accuracy Score : 0.4032

model name : lgb.LGBM
              precision    recall  f1-score   support

           0       0.40      0.20      0.27        20
           1       0.69      0.86      0.77        42


In [8]:
def rate_of_return():
    df['percent'] = round((df.Close-df.Close.shift(1))/df.Close.shift(1)*100, 2)
    for i in range(len(df)-1):
        if (df.loc[i, 'predicted'] == 0):
            df.loc[i+1, 'percent'] = df.loc[i+1, 'percent']

In [9]:
for m in models : 
    model = m['model']
    model.fit(train, train_target)
    
    predicted = model.predict(test)
    
    df = pd.concat([test.reset_index().drop('Date', axis=1), pd.DataFrame(predicted, columns = ['predicted'])], axis=1)
    
    rate_of_return()
    
    df.dropna(inplace = True)

    print('model name : {}'.format(m['name']))
    print('첫날을 제외한 거래일수 : {}'.format(len(df)))
    print('누적 수익률 : {}'.format(round(df['percent'].sum(), 2)))
    print('1일 평균 수익률 : {}\n'.format(round(df['percent'].sum()/(len(df)-1),2)))

model name : Logistic
첫날을 제외한 거래일수 : 61
누적 수익률 : 48.11
1일 평균 수익률 : 0.8

model name : KNN
첫날을 제외한 거래일수 : 61
누적 수익률 : 48.11
1일 평균 수익률 : 0.8

model name : lgb.LGBM
첫날을 제외한 거래일수 : 61
누적 수익률 : 48.11
1일 평균 수익률 : 0.8

model name : NaiveBayes
첫날을 제외한 거래일수 : 61
누적 수익률 : 48.11
1일 평균 수익률 : 0.8



## 1.3 주가 관련 보조지표를 활용하여 등락 예측

In [10]:
# StockInsider 패키지를 사용해 여러가지 주가 관련 보조지표 산출
# 하지만 중국에서 개발된 패키지라 한국에 상장된 종목 코드는 등록되어 있지 않다.
# from_external_csv_data 함수를 이용해 코스피 종목의 주식데이터를 받아서 StockInsider 패키지에 넣어줘야 한다.

def get_stock(code):
    import FinanceDataReader as fdr
    from insider import StockInsider
    import os

    df=fdr.DataReader("code")
    df["day"]=df.index
    df.columns=['open', 'close','high', 'low', 'volumn', "price_change",'day']

    df["ma5"]=df["close"].rolling(5).mean()
    df["ma10"]=df["close"].rolling(10).mean()
    df["ma20"]=df["close"].rolling(20).mean()
    df["v_ma5"]=df["volumn"].rolling(5).mean()
    df["v_ma10"]=df["volumn"].rolling(10).mean()
    df["v_ma20"]=df["volumn"].rolling(20).mean()
    df["percent_change"]=df["close"].pct_change()

    
    if os.path.exists("./data")==False:

        os.mkdir("./data")

    df.to_csv("./data/{}.csv".format(code))
    fpath = "./data/{}.csv".format(code)
    si = StockInsider.from_external_csv_data(fpath=fpath, code=code)

    return si
    
# 주사 데이터는 FinanceDataReader 패키지로 가져오고, 전처리를 후에 csv파일로 저장한다.
# 이 파일을 이용해서 StockInsider 객체를 만든다

In [11]:
import FinanceDataReader as fdr
si=get_stock("03572")
si.plot()

In [12]:
# 이동평균선
def ma(df):
    #이동평균 칼럼 추가 당일 포함 3일 7일 15일 30일
    df['MA_3'] = df['Close'].rolling(window=3).mean() 
    df['MA_7'] = df['Close'].rolling(window=7).mean()
    df['MA_15'] = df['Close'].rolling(window=15).mean() 
    df['MA_30'] = df['Close'].rolling(window=30).mean()

In [13]:
# 지수이동평균선
def ema(df, day):
    df['EWM_{}'.format(day)] = df['Close'].ewm(span=day).mean()
    df['EWM_{}'.format(day)] = df['Close'].ewm(span=day).mean()

In [14]:
ema(stock, 15)

In [15]:
# 이격도 (PPO : Percentage Price oscillator)
def ppo(df, day):
    df['PPO_{}'.format(day)] = (df['Close']/df['MA_{}'.format(day)])*100

In [16]:
# RSI
def U(x):
    if x >= 0:
        return x
    else :
        return 0
    
def D(x):
    if x <= 0:
        return x*(-1)
    else :
        return 0

def rsi(df):
    df['diff_rsi'] = (df['Close'].shift(1)-df['Close'])
    df['AU'] = df['diff_rsi'].apply(U).rolling(window=14).mean() 
    df['AD'] = df['diff_rsi'].apply(D).rolling(window=14).mean() 
    df['RSI'] = df['AU']/(df['AU']+df['AD'])

In [17]:
# rsi(stock)
# stock.drop(['diff_rsi', 'AU', 'AD'], axis=1, inplace = True)

In [18]:
# 모멘텀 스토캐스틱 %K, %D, Fast, Slow
def high_low(day):
    global stock
    stock = stock.reset_index()
    for i in range(len(stock)-day+1):
        stock.loc[i, 'High_st']= stock[i:i+day]['High'].max()
        stock.loc[i, 'Low_st']= stock[i:i+day]['Low'].min()
        
    stock['High_st_4'] = stock['High_st'].shift(4)
    stock['Low_st_4'] = stock['Low_st'].shift(4)
    
    stock['fast_K'] = (stock['Close']-stock['Low_st_4'])/(stock['High_st_4']-stock['Low_st_4'])
    stock['fast_D'] = stock['fast_K'].rolling(3).mean()
    stock['slow_K'] = stock['fast_D']
    stock['slow_D'] = stock['slow_K'].rolling(3).mean()
    stock = stock.set_index('Date')

In [19]:
stock['High_st'] = np.nan
stock['Low_st'] = np.nan
high_low(5)

In [20]:
# CCI
def CCI(df):
    #CCI = (M-N) / (0.015*D)
    # M=특정일의 고가,저가, 종가의 평균
    # N = 일정기간동안의 단순이동평균 통상적으로 20일로 사용
    # D = M-N의 일정기간동안의 단순이동평균
    M = ((df.High)+(df.Low)+(df.Close)) / 3
    N = M.rolling(20).mean()
    D = (M-N).rolling(20).mean()
    CCI = (M - N)/ (0.015 * D)
    stock['CCI'] = CCI

In [21]:
# mackd
def macd(df, short=12, long=26, t=9):
    
    ma_12=df.Close.ewm(span=12).mean()
    ma_26 = df.Close.ewm(span=26).mean() # 장기(26) EMA
    macd = ma_12 - ma_26 # MACD
    macdSignal = macd.ewm(span=9).mean() # Signal
    macdOscillator = macd - macdSignal # Oscillator
    stock['macd'] = macdOscillator

In [22]:
macd(stock)

In [23]:
stock = stock.dropna()

## 1.4 적정 보조지표 찾기

In [24]:
from sklearn import datasets
from sklearn import metrics

# 분류모형
import lightgbm as lgb
logistic = LogisticRegression()
knn = KNeighborsClassifier()
naive = GaussianNB()
lgb = lgb.LGBMClassifier()

models = [{'name' : 'Logistic', 'model' : logistic}, {'name' : 'KNN', 'model' : knn},
          {'name' : 'lgb.LGBM', 'model' : lgb}, {'name' : 'NaiveBayes', 'model' : naive}]

In [25]:
while (True):
    index = []
    i = 0
    
    if i == 0:
        stock_train = stock.iloc[:-1]
        target = stock_train['fluctuation']
        stock_train = stock_train.drop('fluctuation', axis = 1)

        train, test, train_target, test_target = train_test_split(
            stock_train, target, test_size=0.3, shuffle=False)
        
        accuracy = []
        
        for m in models:
            model = m['model']
            model.fit(train, train_target)
    
            predicted = model.predict(test)
    
            score = metrics.accuracy_score(test_target, predicted)
            accuracy.append(score)
        
        init_score = np.mean(accuracy)
        
        print(i, ' iteration, initial score : {}'.format(round(init_score, 4)))
        
        i += 1    
        
    if i == 1:
        stock_2 = stock.copy()
        
        index2 = [ma, ema, ppo]
        
        for dex in index2:
            if dex == ma:
                dex(stock_2)
            if dex == ema:
                dex(stock_2, 15)
                dex(stock_2, 30)
            if dex == ppo:
                dex(stock_2, 15)
                dex(stock_2, 30)
        
        stock_2 = stock_2.dropna()
        stock_2 = stock_2.iloc[len(stock_2)-160:]
        
        today = stock_2.iloc[-1]
        stock_2_df = stock_2.iloc[:-1]
        target = stock_2_df['fluctuation']
        stock_2_df = stock_2_df.drop('fluctuation', axis = 1)

        train, test, train_target, test_target = train_test_split(stock_2_df, target, test_size = 0.3, shuffle=False )
        
        accuracy = []
        
        for m in models:
            
            model = m['model']
            model.fit(train, train_target)
    
            predicted = model.predict(test)
    
            score = metrics.accuracy_score(test_target, predicted)
            accuracy.append(score)
        
        accuracy_score = np.mean(accuracy)
        
        print(i, ' iteration, Accuracy score : {}'.format(round(accuracy_score, 4)))
        
        if init_score <= accuracy_score:
            index = index2
            
        i += 1
        
    
    if i == 2:
        stock_2 = stock.copy()
        
        rsi(stock_2)
        stock_2.drop(['diff_rsi', 'AU', 'AD'], axis=1, inplace = True)
        
        stock_2 = stock_2.dropna()
        stock_2 = stock_2.iloc[len(stock_2)-160:]
        
        today = stock_2.iloc[-1]
        stock_2_df = stock_2.iloc[:-1]
        target = stock_2_df['fluctuation']
        stock_2_df = stock_2_df.drop('fluctuation', axis = 1)

        train, test, train_target, test_target = train_test_split(
            stock_2_df, target, test_size = 0.3, shuffle=False )
        
        accuracy = []
        
        for m in models:
            
            model = m['model']
            model.fit(train, train_target)
    
            predicted = model.predict(test)
    
            score = metrics.accuracy_score(test_target, predicted)
            accuracy.append(score)
        
        accuracy_score = np.mean(accuracy)
        
        print(i, ' iteration, Accuracy score : {}'.format(round(accuracy_score, 4)))
        
        if init_score <= accuracy_score:
            index.append(rsi)
        
        i += 1
        
    if i == 3:
        stock_2 = stock.copy()
        
        stock_2['High_st'] = np.nan
        stock_2['Low_st'] = np.nan
        high_low(5)
        stock_2 = stock_2.drop(['High_st', 'Low_st', 'High_st_4', 'Low_st_4', 'fast_K', 'fast_D'], axis = 1)
        
        stock_2 = stock_2.dropna()
        stock_2 = stock_2.iloc[len(stock_2)-160:]
        
        today = stock_2.iloc[-1]
        stock_2_df = stock_2.iloc[:-1]
        target = stock_2_df['fluctuation']
        stock_2_df = stock_2_df.drop('fluctuation', axis = 1)

        train, test, train_target, test_target = train_test_split(
            stock_2_df, target, test_size = 0.3, shuffle=False )
        
        accuracy = []
        
        for m in models:
            
            model = m['model']
            model.fit(train, train_target)
    
            predicted = model.predict(test)
    
            score = metrics.accuracy_score(test_target, predicted)
            accuracy.append(score)
        
        accuracy_score = np.mean(accuracy)
        
        print(i, ' iteration, Accuracy score : {}'.format(round(accuracy_score, 4)))
        
        if init_score <= accuracy_score:
            index.append(high_low)
           
        i += 1
        
        
    if i == 4:
        stock_2 = stock.copy()
        
        CCI(stock_2)
        
        stock_2 = stock_2.dropna()
        stock_2 = stock_2.iloc[len(stock_2)-160:]
        
        today = stock_2.iloc[-1]
        stock_2_df = stock_2.iloc[:-1]
        target = stock_2_df['fluctuation']
        stock_2_df = stock_2_df.drop('fluctuation', axis = 1)

        train, test, train_target, test_target = train_test_split(
            stock_2_df, target, test_size = 0.3, shuffle=False )
        
        accuracy = []
        
        for m in models:
            
            model = m['model']
            model.fit(train, train_target)
    
            predicted = model.predict(test)
    
            score = metrics.accuracy_score(test_target, predicted)
            accuracy.append(score)
        
        accuracy_score = np.mean(accuracy)
        
        print(i, ' iteration, Accuracy score : {}'.format(round(accuracy_score, 4)))
        
        if init_score <= accuracy_score:
            index.append(CCI)

        break

0  iteration, initial score : 0.5905
1  iteration, Accuracy score : 0.6406
2  iteration, Accuracy score : 0.6146
3  iteration, Accuracy score : 0.5938
4  iteration, Accuracy score : 0.6198


In [26]:
# # stock_2 = stock.copy()
        
# #     for dex in index:
# #         if dex == ma:
# #             dex(stock_2)
# #         if dex == ema:
# #             dex(stock_2, 15)
# #             dex(stock_2, 30)
# #         if dex == ppo:
# #             dex(stock_2, 15)
# #             dex(stock_2, 30)
# #         if dex == rsi:
# #             dex(stock_2)
# #             stock_2.drop(['diff_rsi', 'AU', 'AD'], axis=1, inplace = True)
# #         if dex == high_low:
# #             stock_2['high_st'] = np.nan
# #             stock_2['low_st'] = np.nan
# #             dex(5)
# #             stock_2.drop(['high_st', 'low_st', 'high_st_4', 'low_st_4', 'fast_K', 'fast_D'], axis = 1, inplace = True)
# #         if dex == CCI:
# #             dex(stock_2)
    
        
# #     stock_2 = stock_2.dropna()
# #     stock_2 = stock_2.iloc[len(stock_2)-160:]
        
# #     today = stock_2.iloc[-1]
# #     stock_2_df = stock_2.iloc[:-1]
# #     target = stock_2_df['fluctuation']
# #     stock_2_df = stock_2_df.drop('fluctuation', axis = 1)

# #     train, test, train_target, test_target = train_test_split(stock_2_df, target, test_size = 0.3, shuffle=False )
        
# #     accuracy = []
        
# #     for m in models:
            
# #         model = m['model']
# #         model.fit(train, train_target)
    
# #         predicted = model.predict(test)
    
# #         score = metrics.accuracy_score(test_target, predicted)
# #         accuracy.append(score)
        
# #     accuracy_score = np.mean(accuracy)
        
# #     print('Final iteration, score : {}'.format(round(accuracy_score, 4)))
# #     break

In [27]:
# 분류모형
import lightgbm as lgb
logistic = LogisticRegression()
knn = KNeighborsClassifier()
naive = GaussianNB()
lgb = lgb.LGBMClassifier()

models = [{'name' : 'Logistic', 'model' : logistic}, {'name' : 'KNN', 'model' : knn},
          {'name' : 'lgb.LGBM', 'model' : lgb}, {'name' : 'NaiveBayes', 'model' : naive}]


for m in models : 
    model = m['model']
    model.fit(train, train_target)
    
    predicted = model.predict(test)

#Accuracy : 전체 샘플 중 맞게 예측한 샘플수의 비율
#Precision(정밀도) : postive라고 예측한 것 중에서 실제 postive인 것
#Recall(재현율) : 실제 postive중에 예측한 postive 비율
    print ('model name : {}'.format(m['name']))
    print (metrics.classification_report(test_target, predicted))

#confusion_matrix에서
#행은 실제값, 열은 예측한 값으로 0 1 순서대로 임
    print('Confusion Matrix') 
    print (metrics.confusion_matrix (test_target, predicted))

    print ('Accuracy Score : {:.4f}\n'.format(metrics.accuracy_score(test_target, predicted)))

model name : Logistic
              precision    recall  f1-score   support

           0       0.62      0.72      0.67        18
           1       0.81      0.73      0.77        30

    accuracy                           0.73        48
   macro avg       0.72      0.73      0.72        48
weighted avg       0.74      0.73      0.73        48

Confusion Matrix
[[13  5]
 [ 8 22]]
Accuracy Score : 0.7292

model name : KNN
              precision    recall  f1-score   support

           0       0.40      0.33      0.36        18
           1       0.64      0.70      0.67        30

    accuracy                           0.56        48
   macro avg       0.52      0.52      0.52        48
weighted avg       0.55      0.56      0.55        48

Confusion Matrix
[[ 6 12]
 [ 9 21]]
Accuracy Score : 0.5625

model name : lgb.LGBM
              precision    recall  f1-score   support

           0       0.36      0.22      0.28        18
           1       0.62      0.77      0.69        30


In [28]:
for m in models : 
    model = m['model']
    model.fit(train, train_target)
    
    predicted = model.predict(test)
    
    df = pd.concat([test.reset_index().drop('Date', axis=1), pd.DataFrame(predicted, columns = ['predicted'])], axis=1)
    
    rate_of_return()
    
    df.dropna(inplace = True)

    print('model name : {}'.format(m['name']))
    print('첫날을 제외한 거래일수 : {}'.format(len(df)))
    print('누적 수익률 : {}'.format(round(df['percent'].sum(), 2)))
    print('1일 평균 수익률 : {}\n'.format(round(df['percent'].sum()/(len(df)-1),2)))

model name : Logistic
첫날을 제외한 거래일수 : 47
누적 수익률 : 35.31
1일 평균 수익률 : 0.77

model name : KNN
첫날을 제외한 거래일수 : 47
누적 수익률 : 35.31
1일 평균 수익률 : 0.77

model name : lgb.LGBM
첫날을 제외한 거래일수 : 47
누적 수익률 : 35.31
1일 평균 수익률 : 0.77

model name : NaiveBayes
첫날을 제외한 거래일수 : 47
누적 수익률 : 35.31
1일 평균 수익률 : 0.77



## 2.1 감성분석 데이터를 포함하여 등락 예측

In [29]:
score = pd.read_csv('카카오_감성분석_re.csv')

In [30]:
score_3 = score.copy()
stock_3 = stock_2.copy()

In [31]:
# stock과 score의 갯수가 달라 맞는 date를 찾아 출력하는 함수
def date_range():
    score_num = len(score_3)
    for i in range(1, 30):
        date_num = len(pd.Date_range('9/{}/2020'.format(i), '6/30/2021'))
        if score_num == date_num:
            date = pd.Date_range('9/{}/2020'.format(i), '6/30/2021')
            break
    return date

In [32]:
stock_3 = stock_3.reset_index() # 합치기 위해 날짜 인덱스를 열로 보냄

#stock_3데이터와 score_3데이터를 한 dataframe으로 합치기 위함
stock_data = pd.merge(stock_3, score_3, how='left')
stock_data.dropna(inplace=True)
stock_data = stock_data.set_index('Date') # index를 date로 설정

In [33]:
stock_data.head()

,Open,High,Low,Close,Volume,fluctuation,EWM_15,High_st,Low_st,High_st_4,Low_st_4,fast_K,fast_D,slow_K,slow_D,macd,Pos,Neu,Neg
Date,,,,,,,,,,,,,,,,,,,
2020-11-03,66900,67800,66400,67300,2018930,1,69033.193974,76100.0,66400.0,69300.0,65600.0,0.459459,0.388002,0.388002,0.485503,-231.328152,1,0,0
2020-11-04,68100,72200,68000,71900,7155130,1,69392.698315,76100.0,68000.0,72200.0,65600.0,0.954545,0.622850,0.622850,0.495843,132.138431,0,0,0
2020-11-05,71800,73700,70600,72600,5306835,1,69794.739847,76100.0,69500.0,73700.0,65600.0,0.864198,0.759401,0.759401,0.590084,410.139001,1,0,0
2020-11-06,73400,76100,73100,74100,6577860,1,70334.222745,76100.0,69500.0,76100.0,65600.0,0.809524,0.876089,0.876089,0.752780,668.740607,1,0,0
2020-11-09,75000,75200,73500,74300,2968675,0,70831.012833,75200.0,69500.0,76100.0,66400.0,0.814433,0.829385,0.829385,0.821625,819.551555,0,1,0


In [34]:
len(stock_data)

160

## 2.2 머신러닝 준비

In [35]:
# 정확도가 별반 차이가 없어서 최적화 추가
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
columns = [i for i in stock_data.columns]
stock_data[columns] = scaler.fit_transform(stock_data)

In [36]:
from sklearn.model_selection import train_test_split

stock_df = stock_data.iloc[:-1]
target = stock_df['fluctuation']
stock_df = stock_df.drop('fluctuation', axis = 1)

train, test, train_target, test_target = train_test_split(
    stock_df, target, test_size = 0.3, shuffle=False )

In [37]:
train.head()

,Open,High,Low,Close,Volume,EWM_15,High_st,Low_st,High_st_4,Low_st_4,fast_K,fast_D,slow_K,slow_D,macd,Pos,Neu,Neg
Date,,,,,,,,,,,,,,,,,,
2020-11-03,0.000000,0.000000,0.000000,0.000000,0.109676,0.000000,0.018237,0.000000,0.000000,0.000000,0.459459,0.337170,0.337170,0.427131,0.283311,1.0,0.0,0.0
2020-11-04,0.011418,0.041825,0.016913,0.045010,0.388693,0.004651,0.018237,0.019025,0.027965,0.000000,0.954545,0.608878,0.608878,0.440266,0.362738,0.0,0.0,0.0
2020-11-05,0.046622,0.056084,0.044397,0.051859,0.288287,0.009851,0.018237,0.036861,0.042430,0.000000,0.864198,0.766860,0.766860,0.559977,0.423488,1.0,0.0,0.0
2020-11-06,0.061846,0.078897,0.070825,0.066536,0.357333,0.016830,0.018237,0.036861,0.065574,0.000000,0.809524,0.901862,0.901862,0.766645,0.480000,1.0,0.0,0.0
2020-11-09,0.077069,0.070342,0.075053,0.068493,0.161269,0.023256,0.009119,0.036861,0.065574,0.009535,0.814433,0.847828,0.847828,0.854097,0.512956,0.0,1.0,0.0


In [38]:
test.tail()

,Open,High,Low,Close,Volume,EWM_15,High_st,Low_st,High_st_4,Low_st_4,fast_K,fast_D,slow_K,slow_D,macd,Pos,Neu,Neg
Date,,,,,,,,,,,,,,,,,,
2021-06-17,0.714558,0.776616,0.783298,0.789628,0.293830,0.837625,0.969605,0.881094,0.773385,0.791418,0.914286,0.942047,0.942047,0.996433,0.757806,1.0,0.0,0.0
2021-06-18,0.790676,0.852662,0.878436,0.858121,0.418637,0.871929,1.000000,0.988109,0.850530,0.839094,0.883721,0.897204,0.897204,0.958811,0.843176,1.0,0.0,0.0
2021-06-21,0.843007,0.871673,0.904863,0.858121,0.452241,0.901945,1.000000,1.000000,0.869817,0.892729,0.763158,0.875984,0.875984,0.926813,0.857781,1.0,0.0,0.0
2021-06-22,0.866794,0.866920,0.941860,0.897260,0.253259,0.934677,1.000000,1.000000,0.869817,0.892729,0.973684,0.898891,0.898891,0.911018,0.881854,1.0,0.0,0.0
2021-06-23,0.909610,0.971483,1.000000,1.000000,0.493193,0.980296,1.000000,1.000000,0.971070,0.892729,0.983051,0.937198,0.937198,0.925655,1.000000,0.0,0.0,0.0


## 2.3 모델링

In [39]:
import lightgbm as lgb
logistic = LogisticRegression()
knn = KNeighborsClassifier()
naive = GaussianNB()
lgb = lgb.LGBMClassifier()

models = [{'name' : 'Logistic', 'model' : logistic},
        {'name' : 'KNN', 'model' : knn},
        {'name' : 'lgb.LGBM', 'model' : lgb},
        {'name' : 'NaiveBayes', 'model' : naive}]

for m in models : 
    model = m['model']
    model.fit(train, train_target)
    
    predicted = model.predict(test)

#Accuracy : 전체 샘플 중 맞게 예측한 샘플수의 비율
#Precision(정밀도) : postive라고 예측한 것 중에서 실제 postive인 것
#Recall(재현율) : 실제 postive중에 예측한 postive 비율
    print ('model name : {}'.format(m['name']))
    print (metrics.classification_report(test_target, predicted))

#confusion_matrix에서
#행은 실제값, 열은 예측한 값으로 0 1 순서대로 임
    print('Confusion Matrix') 
    print (metrics.confusion_matrix (test_target, predicted))

    print ('Accuracy Score : {:.4f}\n'.format(metrics.accuracy_score(test_target, predicted)))

model name : Logistic
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        18
         1.0       0.62      1.00      0.77        30

    accuracy                           0.62        48
   macro avg       0.31      0.50      0.38        48
weighted avg       0.39      0.62      0.48        48

Confusion Matrix
[[ 0 18]
 [ 0 30]]
Accuracy Score : 0.6250

model name : KNN
              precision    recall  f1-score   support

         0.0       0.50      0.39      0.44        18
         1.0       0.68      0.77      0.72        30

    accuracy                           0.62        48
   macro avg       0.59      0.58      0.58        48
weighted avg       0.61      0.62      0.61        48

Confusion Matrix
[[ 7 11]
 [ 7 23]]
Accuracy Score : 0.6250

model name : lgb.LGBM
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        18
         1.0       0.62      0.97      0.75        30


In [40]:
for m in models : 
    model = m['model']
    model.fit(train, train_target)
    
    predicted = model.predict(test)
    
    df = pd.concat([test.reset_index().drop('Date', axis=1),
                    pd.DataFrame(predicted, columns = ['predicted'])], axis=1)
    
    rate_of_return()
    
    df.dropna(inplace = True)

    print('model name : {}'.format(m['name']))
    print('첫날을 제외한 거래일수 : {}'.format(len(df)))
    print('누적 수익률 : {}'.format(round(df['percent'].sum(), 2)))
    print('1일 평균 수익률 : {}\n'.format(round(df['percent'].sum()/(len(df)-1),2)))

model name : Logistic
첫날을 제외한 거래일수 : 47
누적 수익률 : 70.48
1일 평균 수익률 : 1.53

model name : KNN
첫날을 제외한 거래일수 : 47
누적 수익률 : 70.48
1일 평균 수익률 : 1.53

model name : lgb.LGBM
첫날을 제외한 거래일수 : 47
누적 수익률 : 70.48
1일 평균 수익률 : 1.53

model name : NaiveBayes
첫날을 제외한 거래일수 : 47
누적 수익률 : 70.48
1일 평균 수익률 : 1.53



In [41]:
stock_data.head()

,Open,High,Low,Close,Volume,fluctuation,EWM_15,High_st,Low_st,High_st_4,Low_st_4,fast_K,fast_D,slow_K,slow_D,macd,Pos,Neu,Neg
Date,,,,,,,,,,,,,,,,,,,
2020-11-03,0.000000,0.000000,0.000000,0.000000,0.109676,1.0,0.000000,0.018237,0.000000,0.000000,0.000000,0.459459,0.337170,0.337170,0.427131,0.283311,1.0,0.0,0.0
2020-11-04,0.011418,0.041825,0.016913,0.045010,0.388693,1.0,0.004651,0.018237,0.019025,0.027965,0.000000,0.954545,0.608878,0.608878,0.440266,0.362738,0.0,0.0,0.0
2020-11-05,0.046622,0.056084,0.044397,0.051859,0.288287,1.0,0.009851,0.018237,0.036861,0.042430,0.000000,0.864198,0.766860,0.766860,0.559977,0.423488,1.0,0.0,0.0
2020-11-06,0.061846,0.078897,0.070825,0.066536,0.357333,1.0,0.016830,0.018237,0.036861,0.065574,0.000000,0.809524,0.901862,0.901862,0.766645,0.480000,1.0,0.0,0.0
2020-11-09,0.077069,0.070342,0.075053,0.068493,0.161269,0.0,0.023256,0.009119,0.036861,0.065574,0.009535,0.814433,0.847828,0.847828,0.854097,0.512956,0.0,1.0,0.0


In [42]:
stock_data.head()

,Open,High,Low,Close,Volume,fluctuation,EWM_15,High_st,Low_st,High_st_4,Low_st_4,fast_K,fast_D,slow_K,slow_D,macd,Pos,Neu,Neg
Date,,,,,,,,,,,,,,,,,,,
2020-11-03,0.000000,0.000000,0.000000,0.000000,0.109676,1.0,0.000000,0.018237,0.000000,0.000000,0.000000,0.459459,0.337170,0.337170,0.427131,0.283311,1.0,0.0,0.0
2020-11-04,0.011418,0.041825,0.016913,0.045010,0.388693,1.0,0.004651,0.018237,0.019025,0.027965,0.000000,0.954545,0.608878,0.608878,0.440266,0.362738,0.0,0.0,0.0
2020-11-05,0.046622,0.056084,0.044397,0.051859,0.288287,1.0,0.009851,0.018237,0.036861,0.042430,0.000000,0.864198,0.766860,0.766860,0.559977,0.423488,1.0,0.0,0.0
2020-11-06,0.061846,0.078897,0.070825,0.066536,0.357333,1.0,0.016830,0.018237,0.036861,0.065574,0.000000,0.809524,0.901862,0.901862,0.766645,0.480000,1.0,0.0,0.0
2020-11-09,0.077069,0.070342,0.075053,0.068493,0.161269,0.0,0.023256,0.009119,0.036861,0.065574,0.009535,0.814433,0.847828,0.847828,0.854097,0.512956,0.0,1.0,0.0


In [43]:
# # statsmodels 패키지는 추정 및 검정, 회귀분석, 시계열분석 등의 기능을 제공하는 파이썬 패키지
# # 기존에 R에서 가능했던 다양한 회귀분석과 시계열분석 방법론을 그대로 파이썬에서 이용 가능
# import statsmodels.api as sm

# logis = sm.Logit(stock_data['fluctuation'],stock_data[stock_df.columns])
# result = logis.fit()

# result.summary()